<a href="https://colab.research.google.com/github/nyanta012/chatgpt_api_practice/blob/main/section6/section6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratoryのショートカットキー
1. **現在のセルを実行し、次のセルを選択**: `Shift + Enter`
2. **コードセルをテキストセルにする**: `Ctrl + M M`
3. **コードセルを上に追加**: `Ctrl + M A`
4. **コードセルを下に追加**: `Ctrl + M B`
5. **セルの削除**: `Ctrl + M D`

# API KEYの設定

In [ ]:
%%capture
!pip install openai==0.28.1

In [ ]:
import getpass
import json

import openai
from IPython.display import Markdown, display

apikey = getpass.getpass(prompt="OpenAIのAPIキーを入力してください")
openai.api_key = apikey

OpenAIのAPIキーを入力してください··········


# 並行処理で高速に回答を得よう

並行処理とは簡単に言うと、並行して異なる処理を行うこと

**例えば、料理を作るケースを考えると･･･**
　　何かの材料をオーブンで温めている待ち時間に、別の材料を切ったりするイメージ

**今回の例だと･･･**
　　APIのリクエストを送ってから返答を得るまで(オーブンで温めている待ち時間)
　　次のAPIのリクエストを送る(別の材料を切ったりする)と高速化できる

In [ ]:
def get_chatgpt_response(
    user_input: str,
    template: str,
    model: str = "gpt-3.5-turbo",
    temperature: float = 0,
    max_tokens: int = 500,
):
    """
    ChatGPTに対して対話を投げかけ、返答を取得する
    """
    prompt = template.format(user_input=user_input)
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [ ]:
PROMPT_TEMPLATE = """
次の文章を英語にしてください

{user_input}

"""

In [ ]:
%%time
get_chatgpt_response("こんにちは", PROMPT_TEMPLATE)

CPU times: user 30.6 ms, sys: 4.85 ms, total: 35.4 ms
Wall time: 933 ms


'Hello.'

In [ ]:
from concurrent.futures import ThreadPoolExecutor

In [ ]:
%%time

# スレッドプールを作成し、最大3つのスレッドを同時に実行できるようにする
list_user_input = ["こんにちは", "こんばんは", "おはよう"]
tpe = ThreadPoolExecutor(max_workers=len(list_user_input))

list_response = []
for user_input in list_user_input:
    response = tpe.submit(get_chatgpt_response, user_input, PROMPT_TEMPLATE)
    list_response.append(response)
tpe.shutdown()  # 全てのタスクが実行されるとスレッドが終了する

# 結果を表示
for r in list_response:
    print(r.result())

Hello.
Good evening.
Good morning.
CPU times: user 54.9 ms, sys: 7.17 ms, total: 62.1 ms
Wall time: 904 ms


# Section5のメール生成を並行処理で実装してみよう

In [ ]:
def get_chatgpt_response(
    user_input: str,
    category_1: str,
    category_2: str,
    template: str,
    model: str = "gpt-3.5-turbo",
    temperature: float = 0,
    max_tokens: int = 500,
):
    """
    ChatGPTに対して対話を投げかけ、返答を取得する
    """
    prompt = template.format(
        user_input=user_input, category_1=category_1, category_2=category_2
    )
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [ ]:
PROMPT_TEMPLATE = """
    あなたはカスタマーセンターの従業員です。
    あなたの仕事は顧客の問い合わせに対して、適切な部門の連絡先を教えることです。
    顧客の問い合わせ内容とカテゴリ－は下記の####で区切られた内容です。

    ####
    {user_input}

    第一カテゴリー:{category_1}
    第二カテゴリー:{category_2}
    ####

   下記ステップに従って、メールを作成してください。

    1. 第一カテゴリーと第二カテゴリーに応じてメールアドレスを下記から抽出してください。
        製品：スペック は product_spec@example.com
        製品：値段 は product_price@example.com
        製品：購入方法 は product_purchase_method@example.com
        注文：注文状況 は order_status@example.com
        注文：注文変更 は order_change@example.com
        注文：支払い方法 は order_payment_method@example.com
        配送：配送ステータス は delivery_status@example.com
        配送：配送オプション は delivery_option@example.com
        配送：配送問題 は delivery_issue@example.com

    2. メール文章を作成
       ユーザーからの問い合わせ内容に対して1で抽出したメールアドレスに送信するようにお願いする文章を書いてください。
        形式：
          件名

          文章
    """

In [ ]:
import pandas as pd

In [ ]:
questions = [
    "このカメラの最大シャッタースピードは何ですか？",
    "私のパッケージの追跡番号を教えてください。",
    "エクスプレス配送は可能ですか？",
    "特定の日時に配送することはできますか？",
    "配送された商品が損傷していました。どうすればいいですか？",
    "注文の支払いにApple PayやGoogle Payは使用できますか？",
    "私のパッケージはいつ到着予定ですか？",
    "注文の支払いにクレジットカードは使用できますか？",
    "私の注文はまだ出荷されていませんか？",
    "この製品はオンラインで注文することができますか？",
]

list_json = [
    {"key": "製品", "value": "スペック"},
    {"key": "配送", "value": "配送ステータス"},
    {"key": "配送", "value": "配送オプション"},
    {"key": "配送", "value": "配送オプション"},
    {"key": "配送", "value": "配送問題"},
    {"key": "注文", "value": "支払い方法"},
    {"key": "配送", "value": "配送ステータス"},
    {"key": "注文", "value": "支払い方法"},
    {"key": "配送", "value": "配送ステータス"},
    {"key": "注文", "value": "購入方法"},
]

df = pd.DataFrame(list_json)
df["question"] = questions
df.columns = ["category_1", "category_2", "question"]

In [ ]:
%%time
# スレッドプールを作成
tpe = ThreadPoolExecutor(max_workers=len(df))

list_response = []
for _, row in df.iterrows():
    category_1, category_2, user_question = row['category_1'], row['category_2'], row['question']
    response = tpe.submit(
        get_chatgpt_response, user_question, category_1, category_2, PROMPT_TEMPLATE
    )
    list_response.append(response)
tpe.shutdown()  # 全てのタスクが実行されるとスレッドが終了する


# 結果を表示
for i, r in enumerate(list_response):
    user_question = df.loc[i, "question"]
    display(
        Markdown(
            f"""\n\n### メール {i+1} \n **問い合わせ内容:** {user_question}\n\n **生成文:**\n\n {r.result()}"""
        )
    )



### メール 1 
 **問い合わせ内容:** このカメラの最大シャッタースピードは何ですか？

 **生成文:**

 件名：カメラの最大シャッタースピードについてのお問い合わせ

product_spec@example.com 宛に送信してください。

尊敬する product_spec@example.com 様、

お世話になっております。カスタマーセンターの従業員です。

お客様から、カメラの最大シャッタースピードについてのお問い合わせがありました。

お手数ですが、下記の問い合わせに対してご回答いただけますようお願い申し上げます。

問い合わせ内容：
このカメラの最大シャッタースピードは何ですか？

何卒、よろしくお願い申し上げます。

敬具

カスタマーセンターの従業員



### メール 2 
 **問い合わせ内容:** 私のパッケージの追跡番号を教えてください。

 **生成文:**

 件名：配送ステータスに関するお問い合わせ

お客様

この度は弊社にお問い合わせいただき、誠にありがとうございます。
お客様からのお問い合わせにつきまして、配送ステータスに関するお問い合わせであることを確認いたしました。

下記のメールアドレスにお問い合わせ内容をご連絡いただけますようお願い申し上げます。

配送：配送ステータス は delivery_status@example.com

何卒よろしくお願い申し上げます。

敬具

カスタマーセンターの従業員



### メール 3 
 **問い合わせ内容:** エクスプレス配送は可能ですか？

 **生成文:**

 件名：配送オプションに関するお問い合わせ

お客様からのお問い合わせありがとうございます。
配送オプションに関するお問い合わせについては、以下のメールアドレスにお問い合わせください。

配送：配送オプション は delivery_option@example.com

何かご不明な点がございましたら、お気軽にお問い合わせください。
今後ともよろしくお願いいたします。

敬具
カスタマーセンター



### メール 4 
 **問い合わせ内容:** 特定の日時に配送することはできますか？

 **生成文:**

 件名：配送オプションに関するお問い合わせ

お客様

いつも当社のサービスをご利用いただきありがとうございます。
お問い合わせいただきました配送オプションに関するご質問について、適切な部署にお問い合わせいただくようお願い申し上げます。

配送に関するお問い合わせは、以下のメールアドレスにお送りください。
配送：配送オプション は delivery_option@example.com

何かご不明な点がございましたら、お気軽にお問い合わせください。

よろしくお願いいたします。

カスタマーセンター



### メール 5 
 **問い合わせ内容:** 配送された商品が損傷していました。どうすればいいですか？

 **生成文:**

 件名：配送問題に関するお問い合わせ

お客様

この度は、弊社の商品をご購入いただきありがとうございます。
お客様からのお問い合わせについて、適切な部署にご連絡いただくために、下記のメールアドレスにお問い合わせ内容をお送りください。

配送：配送問題 は delivery_issue@example.com

お手数をおかけいたしますが、何卒よろしくお願い申し上げます。

敬具

カスタマーセンターの従業員

...



### メール 7 
 **問い合わせ内容:** 私のパッケージはいつ到着予定ですか？

 **生成文:**

 件名：配送ステータスに関するお問い合わせについて

お客様へ

この度は弊社製品をご購入いただき、誠にありがとうございます。
お問い合わせいただきました配送ステータスに関しましては、下記のメールアドレスにお問い合わせください。

配送：配送ステータス は delivery_status@example.com

上記メールアドレスにお問い合わせいただけますよう、お願い申し上げます。
何かご不明な点がございましたら、お気軽にお問い合わせください。

敬具



### メール 8 
 **問い合わせ内容:** 注文の支払いにクレジットカードは使用できますか？

 **生成文:**

 件名：注文の支払いについて

product_payment_method@example.com 宛に送信してください。

文章：
お客様からのお問い合わせについて、注文の支払い方法に関するご質問がありました。
この件については、order_payment_method@example.com にお問い合わせいただけます。
お手数ですが、上記メールアドレスにご連絡いただけますようお願い申し上げます。
何かご不明な点がございましたら、お気軽にお問い合わせください。
よろしくお願いいたします。



### メール 9 
 **問い合わせ内容:** 私の注文はまだ出荷されていませんか？

 **生成文:**

 件名：配送ステータスに関するお問い合わせについて

お客様のお問い合わせありがとうございます。
配送ステータスに関するお問い合わせにつきましては、下記のメールアドレスにご連絡いただけます。

配送：配送ステータス は delivery_status@example.com

何卒よろしくお願いいたします。



### メール 10 
 **問い合わせ内容:** この製品はオンラインで注文することができますか？

 **生成文:**

 件名：オンライン注文に関するお問い合わせ

product_purchase_method@example.com 宛に送信してください。

尊敬するお客様、

この度は弊社製品にご興味をお持ちいただき、誠にありがとうございます。
お問い合わせいただきました「オンラインでの注文方法について」につきまして、
お答えいたします。

オンラインでの注文は可能です。詳細につきましては、下記のメールアドレスにお問い合わせください。

product_purchase_method@example.com

何かご不明な点がございましたら、お気軽にお問い合わせください。

敬具

CPU times: user 229 ms, sys: 9.96 ms, total: 239 ms
Wall time: 7.5 s
